### **BRONZE TO SILVER ENGINE**

### **Parámetros**

In [18]:
# Nombre de la tabla destino
table_name = "h_transactions"
# Nombre de la cuenta de almacenamiento ADLS Gen2. Ej: dlsmde01albego
storage_account_name = "SUSTITUIR"

#### **Proceso**

01. Definición de variables y rutas de acceso.

In [19]:
from delta.tables import DeltaTable
from pyspark.sql.functions import current_date

# Definir variables
data_lake_container = f'abfss://datalake@{storage_account_name}.dfs.core.windows.net'
bronze_folder = 'bronze'    # Carpeta de ingesta de datos (raw)
silver_folder = 'silver'    # Carpeta donde se almacenarán las tablas Delta

# Determinar la ruta de los archivos de origen
source_path = f"{data_lake_container}/{bronze_folder}/"

# Determinar la ruta de la tabla Delta
delta_table_path = f"{data_lake_container}/{silver_folder}/{table_name}"

2. Lectura de archivos de la capa bronze y creación de nuevo campo de fecha de carga del dato (load_date).

In [21]:
# Leer archivo(s) en DataFrame de Spark
sdf = spark.read.format('csv').option("sep", ",").option("encoding", "ISO-8859-1").option("header", "true").load(source_path)

# Añadir una columna con la fecha de carga 
sdf = sdf.withColumn("load_date", current_date())

sdf.show(10)

3. Creación del fichero en Delta en ADLS Gen2 capa silver.

In [22]:
# Crear nueva tabla Delta con los datos nuevos
print('Creating new Delta table...')
sdf.write.format('delta').mode("overwrite").save(delta_table_path)

# Leer la tabla Delta y mostrar su contenido
delta_df = spark.read.format("delta").load(delta_table_path)
print('Showing content of the Delta table...')
delta_df.show(10)

4. Creación de la base de datos silver y una tabla externa delta utilizando la localización del fichero del punto anterior. 

In [23]:
# Crear base de datos Silver si no existe
spark.sql('CREATE DATABASE IF NOT EXISTS Silver')

# Añadir la tabla Delta a la base de datos Silver para facilitar consultas
spark.sql(f'CREATE TABLE IF NOT EXISTS Silver.{table_name} USING DELTA LOCATION \'{delta_table_path}\'')

# Consultar el recuento de filas en la tabla
count_df = spark.sql(f'SELECT COUNT(*) FROM Silver.{table_name}')

5. Consultar la información del resultado.

In [26]:
# Mostrar el recuento en pantalla
print('Showing row count...')
count_df.show()

# Consultar y mostrar los primeros 10 registros de la tabla
df = spark.sql(f'SELECT * FROM Silver.{table_name}')
print('Showing first 10 records...')
df.show(10, truncate=False)

In [ ]:
mssparkutils.session.stop()